<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/A/dataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
!git clone https://github.com/roshantac/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 47 (delta 5), reused 47 (delta 5), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-05-05 14:41:29--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.20.116
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.20.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  24.1MB/s    in 7.5s    

2020-05-05 14:41:37 (22.0 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp -r '/content/gdrive/My Drive/data15' '/content/DenseDepth/'

In [0]:
%cd DenseDepth

In [0]:
!pwd

/content/DenseDepth


In [0]:
import cv2
import os
import numpy as np
import shutil 

imgHt = 480
imgWdth = 720

fnlHt = 240
fnlWdth = 320
global count 
fgbgDir = ""
fgbgmaskDir =""
errCont =0
def processing(bckImg,fgimg,maskimg):
	
	bck = cv2.imread(bckImg,cv2.IMREAD_COLOR)
	fg_img = cv2.imread(fgimg)
	fg_mask =cv2.imread(maskimg,0)
	bck=cv2.resize(bck, (imgWdth,imgHt), interpolation = cv2.INTER_AREA) 
	Srow,Scol,chanel = fg_img.shape
	if((Srow > imgHt) or (Scol > imgWdth)):
		fg_img = cv2.resize(fg_img, (int(fg_img.shape[0]/2), int(fg_img.shape[1]/2)), interpolation = cv2.INTER_AREA) 
		fg_mask = cv2.resize(fg_mask, (int(fg_mask.shape[0]/2), int(fg_mask.shape[1]/2)), interpolation = cv2.INTER_AREA) 
		Srow,Scol,chanel = fg_img.shape

	heighFactor,widthFactor = int((bck.shape[0] - fg_img.shape[0])/3), int((bck.shape[1] - fg_img.shape[1])/4)
	height = 0
	width = 0
	global count
	for i in range(0,4):
		width = 0	
		for j in range(0,5): #
			bck = cv2.imread(bckImg)
			bcktemp=cv2.resize(bck, (imgWdth,imgHt), interpolation = cv2.INTER_AREA) 
			# print(" heighFactor :",height, " widthFactor :", width)
			roi = bcktemp[height:Srow+height,width:Scol+width]
			inv_mask = cv2.bitwise_not(fg_mask)
			# print("mask",fg_mask.shape)
			img_bck2 = cv2.bitwise_and(roi,roi, mask = inv_mask)#after substraction
			img_fg2 = cv2.bitwise_and(fg_img,fg_img, mask = fg_mask) #forground after substraction
			out_img = cv2.add(img_bck2,img_fg2)
			bcktemp[height:Srow+height,width:Scol+width] = out_img
			newImg = np.zeros(bcktemp.shape[:2],fg_mask.dtype)
			newImg[height:Srow + height,width:Scol + width] = fg_mask
			imgfileName = fgbgDir + 'img' + str(count) + '.jpg' 
			cv2.imwrite(imgfileName, cv2.resize(bcktemp, (fnlWdth,fnlHt), interpolation = cv2.INTER_AREA),[int(cv2.IMWRITE_JPEG_QUALITY), 20])
			mskfileName = fgbgmaskDir + 'mask' + str(count) + '.jpg' 
			cv2.imwrite(mskfileName, cv2.resize(newImg, (fnlWdth,fnlHt), interpolation = cv2.INTER_AREA),[int(cv2.IMWRITE_JPEG_QUALITY), 25])
			width = width +  widthFactor
			count +=1

		height = height + heighFactor 


try:
	os.mkdir("Output")
except :
	shutil.rmtree("Output")
	os.mkdir("Output")

os.mkdir("Output/fgbg")
os.mkdir("Output/fgbgmask")

fgbgDir = "Output/fgbg/"
fgbgmaskDir = "Output/fgbgmask/"
count = 0
bckDir = "data15/Bck_Images/"
fgDir = "data15/fg_Images/"
fgmskDir = "data15/fg_mask/" 
imgCount =0
for _, bckfilename in enumerate(os.listdir(bckDir)): 
    bgimage = bckDir+ bckfilename 
    imgCount = imgCount + 1
    for _, fgfilename in enumerate(os.listdir(fgDir)): 
    	fgimage = fgDir + fgfilename
    	fgmask = fgmskDir + fgfilename
    	print("BackImgCount:",imgCount)
    	print("count:",count)
    	processing(bgimage,fgimage,fgmask)
print("errorcases: ",errCont)


Streaming output truncated to the last 5000 lines.
BackImgCount: 178
count: 358060
BackImgCount: 178
count: 358080
BackImgCount: 178
count: 358100
BackImgCount: 178
count: 358120
BackImgCount: 178
count: 358140
BackImgCount: 178
count: 358160
BackImgCount: 178
count: 358180
BackImgCount: 178
count: 358200
BackImgCount: 178
count: 358220
BackImgCount: 178
count: 358240
BackImgCount: 178
count: 358260
BackImgCount: 178
count: 358280
BackImgCount: 178
count: 358300
BackImgCount: 178
count: 358320
BackImgCount: 178
count: 358340
BackImgCount: 178
count: 358360
BackImgCount: 178
count: 358380
BackImgCount: 178
count: 358400
BackImgCount: 178
count: 358420
BackImgCount: 178
count: 358440
BackImgCount: 178
count: 358460
BackImgCount: 178
count: 358480
BackImgCount: 178
count: 358500
BackImgCount: 178
count: 358520
BackImgCount: 178
count: 358540
BackImgCount: 178
count: 358560
BackImgCount: 178
count: 358580
BackImgCount: 178
count: 358600
BackImgCount: 178
count: 358620
BackImgCount: 178
cou

In [0]:
# importing required modules 
from zipfile import ZipFile 
import os 
  
def get_all_file_paths(directory): 
  
    # initializing empty file paths list 
    file_paths = [] 
  
    # crawling through directory and subdirectories 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            # join the two strings in order to form the full filepath. 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
  
    # returning all file paths 
    return file_paths         
  
def main(): 
    # path to folder which needs to be zipped 
    directory = 'Output'
  
    # calling function to get all file paths in the directory 
    file_paths = get_all_file_paths(directory) 
  
    # printing the list of all files to be zipped 
    print('Following files will be zipped:') 
    for file_name in file_paths: 
        print(file_name) 
  
    # writing files to a zipfile 
    with ZipFile('dataSet.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 
  
    print('All files zipped successfully!')  
main()

Streaming output truncated to the last 5000 lines.
Output/fgbgmask/mask84259.jpg
Output/fgbgmask/mask145166.jpg
Output/fgbgmask/mask254782.jpg
Output/fgbgmask/mask134946.jpg
Output/fgbgmask/mask148824.jpg
Output/fgbgmask/mask329713.jpg
Output/fgbgmask/mask392976.jpg
Output/fgbgmask/mask34267.jpg
Output/fgbgmask/mask45690.jpg
Output/fgbgmask/mask139547.jpg
Output/fgbgmask/mask85221.jpg
Output/fgbgmask/mask252240.jpg
Output/fgbgmask/mask8570.jpg
Output/fgbgmask/mask6140.jpg
Output/fgbgmask/mask359450.jpg
Output/fgbgmask/mask316259.jpg
Output/fgbgmask/mask245055.jpg
Output/fgbgmask/mask156369.jpg
Output/fgbgmask/mask203405.jpg
Output/fgbgmask/mask275620.jpg
Output/fgbgmask/mask85948.jpg
Output/fgbgmask/mask146396.jpg
Output/fgbgmask/mask65581.jpg
Output/fgbgmask/mask309381.jpg
Output/fgbgmask/mask47319.jpg
Output/fgbgmask/mask404421.jpg
Output/fgbgmask/mask275885.jpg
Output/fgbgmask/mask102339.jpg
Output/fgbgmask/mask300684.jpg
Output/fgbgmask/mask258568.jpg
Output/fgbgmask/mask138654.jpg

In [0]:
#!cd DenseDepth; python test.py --input '/content/DenseDepth/*.jpg'

In [0]:
!cp '/content/DenseDepth/dataSet.zip' '/content/gdrive/My Drive/'

In [0]:
%matplotlib inline

In [0]:
!python test.py #--input '/content/DenseDepth/Images/*.jpg' 

Using TensorFlow backend.
Loading model...

Model loaded (nyu.h5).

Loaded (6) images of size (480, 640, 3).
shape:  (6, 240, 320, 1)
shape2:  (240, 320, 1)
<Figure size 1000x600 with 1 Axes>
<Figure size 1000x600 with 1 Axes>
final shape (240, 320)
<Figure size 1000x500 with 1 Axes>
<Figure size 1000x500 with 1 Axes>
